In [1]:
import pandas as pd
from pathlib import Path

In [14]:
index_no = 1

In [20]:
df_old = pd.read_csv(f"labels_{index_no}_old.csv", dtype=str)
print(df_old.shape)
df_old.head()

(3280, 5)


,id,pattern,update_date,label,para
0,2203.05084,data,NaN,NaN,composability can also be obtained in terms of...
1,2109.1263,data repos,04/22/22,0,Imperial endorsements were employed as a celeb...
2,2109.1263,data,NaN,NaN,"Before we go towards social media mining, ﬁrst..."
3,2109.1263,data repos,NaN,NaN,"coerce, sadness, and anger, and fear) [86][38]..."
4,2105.00775,"open-source, data repos, code, code available,...",04/22/22,3,"production, wake topology, and propulsive perf..."


In [21]:
df_old['label'].unique()

array([nan, '0', '3', '2'], dtype=object)

In [22]:
df_new = pd.read_csv(f"labels_{index_no}_new.csv", dtype=str).drop(columns=["update_date", "label"])
print(df_new.shape)
df_new.head()

(3065, 4)


,id,pattern,token_count,para
0,1401.8008,"code package, github, code available, code, pa...",56,So we can solve the dual comparison problem (1...
1,1401.8008,data,2,training data
2,1401.8008,data,7,3.2. Kernelized QP for non-separable data
3,1401.8008,data,11,4. Comparison to SVMrank in sushi and simulate...
4,1401.8008,data,15,"Overall from the sushi data, it is clear that ..."


In [23]:
df_new = df_new.merge(df_old, on=["id", "para"], how="left").rename(columns={"pattern_x": "pattern"}).drop(columns=["pattern_y"])
# df_new = df_new[["id", "pattern", "token_count", "update_date", "label", "para"]].sort_values(by=["id", "pattern", "token_count"])
# print(df_new.shape)
# df_new.head()


In [24]:
df_new.head()

,id,pattern_x,token_count,para,pattern_y,update_date,label
0,1401.8008,"code package, github, code available, code, pa...",56,So we can solve the dual comparison problem (1...,"package, code available, code, code package, g...",NaN,NaN
1,1401.8008,data,2,training data,data,NaN,NaN
2,1401.8008,data,7,3.2. Kernelized QP for non-separable data,data,NaN,NaN
3,1401.8008,data,11,4. Comparison to SVMrank in sushi and simulate...,data,NaN,NaN
4,1401.8008,data,15,"Overall from the sushi data, it is clear that ...",data,NaN,NaN


In [44]:
df_new.to_csv(f"labels_{index_no}_new_merged.csv", index=False)

In [4]:
def merge_existing_new_labels(df_existing, df_new):
    df_existing = df_existing.merge(
        df_new[["para", "id", "pattern", "token_count"]], on=["para", "id", "token_count"], how="outer"
    )

    # if pattern_y is NaN, then copy pattern_x to pattern_y
    df_existing["pattern"] = df_existing["pattern_y"].fillna(df_existing["pattern_x"])

    # drop columns that are not needed, pattern_x and pattern_y
    df_existing = df_existing.drop(["pattern_x", "pattern_y"], axis=1)
    return df_existing[["id", "pattern", "token_count", "update_date", "label", "para"]]
